# SQL-запросы. Книжный онлайн-сервис

## 1. Обзор таблиц

**Импорты**

In [1]:
import pandas as pd
from sqlalchemy import create_engine

**Подключение к БД**

In [2]:
db_config = {'user': # имя пользователя, 
             'pwd': # пароль, 
             'host': #host,
             'port': # порт подключения,
             'db': # название базы данных} 

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db']) 

In [4]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

**Таблица книг**

In [5]:
query = "SELECT * FROM books"
books = pd.io.sql.read_sql(query, con = engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


У нас данные о 1000 книг. Пропусков нет.

**Таблица авторов**

In [7]:
query = "SELECT * FROM authors"
authors = pd.io.sql.read_sql(query, con = engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


636 авторов

**Таблица издательств**

In [9]:
query = "SELECT * FROM publishers"
publishers = pd.io.sql.read_sql(query, con = engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [10]:
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


340 издательств

**Таблица пользовательских оценок**

In [11]:
query = "SELECT * FROM ratings"
ratings = pd.io.sql.read_sql(query, con = engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [12]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


6456 оценок от пользователей

**Пользовательские отзывы**

In [13]:
query = "SELECT * FROM reviews"
reviews = pd.io.sql.read_sql(query, con = engine)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [14]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


2793 отзывов. Получается, что отзывы имеют только 43% оценок.

## 2. Запросы и анализ данных

### Количество книг, вышедших после 1 января 2000 года

In [15]:
query = '''

  SELECT COUNT(*) 
  FROM books
  WHERE publication_date > '2000-01-01'

  '''

In [16]:
print('В нашей базе {} книг, вышедших после 1 января 2000 года'.format(pd.io.sql.read_sql(query, con = engine).iloc[0,0]))

В нашей базе 819 книг, вышедших после 1 января 2000 года


Соответственно, 181 книга из нашей базы была выпущена до 1 января 2000 года. 

### Количество обзоров и средняя оценка по каждой книге

Не просто посчитаем количество, а выведем сначала топ-5 по средней оценке, а затем топ-5 по количеству обзоров. 

In [28]:
query = '''

  SELECT books.book_id,
         books.title, 
         ratings.avg_rating,
         reviews.reviews_count
  FROM books
  LEFT JOIN (SELECT book_id,
                    AVG(rating) as avg_rating
             FROM ratings
             GROUP BY book_id) as ratings ON books.book_id = ratings.book_id
  LEFT JOIN (SELECT book_id,
                    COUNT (review_id) as reviews_count
                    FROM reviews
                    GROUP BY book_id) as reviews ON books.book_id = reviews.book_id
  ORDER BY ratings.avg_rating DESC
  '''

Топ-5 по средней оценке

In [29]:
pd.io.sql.read_sql(query, con = engine)

,book_id,title,avg_rating,reviews_count
0,518,Pop Goes the Weasel (Alex Cross #5),5.00,2.0
1,732,The Ghost Map: The Story of London's Most Terr...,5.00,2.0
2,347,In the Hand of the Goddess (Song of the Liones...,5.00,2.0
3,610,Tai-Pan (Asian Saga #2),5.00,2.0
4,330,How to Be a Domestic Goddess: Baking and the A...,5.00,1.0
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,2.25,3.0
996,371,Junky,2.00,2.0
997,316,His Excellency: George Washington,2.00,2.0
998,202,Drowning Ruth,2.00,3.0


Но, откровенно говоря, эти данные нам мало что дают. По такому принципу в топ может попасть любая книга, у которой есть всего одна оценка, но пятерка. Это не совсем объективно.  

In [19]:
query = '''

  SELECT books.book_id,
         books.title, 
         ratings.avg_rating,
         reviews.reviews_count
  FROM books
  LEFT JOIN (SELECT book_id,
                    AVG(rating) as avg_rating
             FROM ratings
             GROUP BY book_id) as ratings ON books.book_id = ratings.book_id
  LEFT JOIN (SELECT book_id,
                    COUNT (review_id) as reviews_count
                    FROM reviews
                    GROUP BY book_id) as reviews ON books.book_id = reviews.book_id
  WHERE reviews.reviews_count != 'NaN'::NUMERIC
  ORDER BY reviews.reviews_count DESC
  '''

Топ-5 по количеству отзывов

In [20]:
pd.io.sql.read_sql(query, con = engine).head()

,book_id,title,avg_rating,reviews_count
0,948,Twilight (Twilight #1),3.662500,7
1,656,The Book Thief,4.264151,6
2,497,Outlander (Outlander #1),4.125000,6
3,963,Water for Elephants,3.977273,6
4,696,The Da Vinci Code (Robert Langdon #2),3.830508,6


С отзывами интереснее. Можно сказать, что это самые обсуждаемые книги. На первом месте - "Сумерки".

### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [21]:
query = '''

  SELECT publishers.publisher,
         COUNT (DISTINCT books.book_id) as books_count
  FROM publishers
  LEFT JOIN books ON publishers.publisher_id = books.publisher_id
  WHERE books.num_pages > 50
  GROUP BY publishers.publisher_id
  ORDER BY books_count DESC
  LIMIT 5

  '''

In [22]:
pd.io.sql.read_sql(query, con = engine)

,publisher,books_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19


Наибольшее число книг выпустило издательство "Penguin Books"

### Автор с самой высокой средней оценкой книг

Учитывая только книги с 50 и более оценками

In [23]:
query = '''

  SELECT authors.author as author,
         AVG (ratings.avg_rating) as avg_rating
  FROM authors
  LEFT JOIN books ON authors.author_id = books.author_id
  LEFT JOIN 
    (SELECT book_id,
            AVG (rating) AS avg_rating,
            COUNT (rating_id) AS rating_counts
     FROM ratings
     GROUP BY book_id
     HAVING COUNT (rating_id) > 49) as ratings ON books.book_id = ratings.book_id
  GROUP BY author
  HAVING AVG (ratings.avg_rating) != 'NaN'
  ORDER BY avg_rating DESC
  
  '''

In [24]:
pd.io.sql.read_sql(query, con = engine).head()

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


На первом месте видим хорошо знакомую всем Джоан Роулинг. Второе имя рядом с ней - это иллюстратор, которая нарисовала иллюстрации к книгам о Гарри Поттере. 

### Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [25]:
query = '''

  SELECT AVG (reviews_count) as avg_reviews_count
  FROM ( 
        SELECT username,
        COUNT(review_id) as reviews_count
        FROM reviews
        GROUP BY username
        HAVING username IN (
                            SELECT username
                            FROM ratings
                            GROUP BY username
                            HAVING COUNT (rating_id) > 50
                            )
                            
                            
        ) as reviews
  
  '''



In [26]:
print('Пользователи, поставившие больше 50 оценок, в среднем пишут по {:.0f} обзора' \
      .format(pd.io.sql.read_sql(query, con = engine).iloc[0,0].round()))

Пользователи, поставившие больше 50 оценок, в среднем пишут по 24 обзора


## Выводы

В этом проекте мы изучили информацию о книгах и сопутствующие данные. Вот результаты.

**В нашей базе:**
- 1000 книг
- от 636 авторов
- и 340 издательств
- по которым мы имеем 6456 оценок от пользователей
- и 2793 пользовательских отзыва

**Результаты анализа:** 
- 819 книг из нашей базы выпущено в этом веке, и, соответственно, 181 - в прошлом. 
- Больше всего книг (без учета небольших брошюр) выпустило издательство "Penguin Books"
- Самые обсуждаемые книги (по количеству пользовательских обзоров) - Сумерки, Книжный вор и Чужестранка. 
- А самые активные пользователи (поставившие больше 50 оценок) пишут в среднем по 24 обзора (1 обзор на 2 книги)
- Самые высокооцененные авторы: Джоан Роулинг, Маркус Зусак и Джон Толкиен